<a href="https://colab.research.google.com/github/Pouyazarbipour/GEE_Bath_Oman/blob/main/GEEpy_OceanCurrent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import ee
import geemap
import xarray as xr
import xee
import numpy as np
import pandas as pd

# احراز هویت و مقداردهی اولیه
ee.Authenticate()
ee.Initialize(project='ee-pwuyaz', opt_url='https://earthengine-highvolume.googleapis.com')

# تعریف منطقه مورد نظر (دریای عمان: 22-26°N، 56-62°E)
geometry = ee.Geometry.Rectangle([56, 22, 62, 26])

In [5]:


# بارگذاری مجموعه داده باتیمتری (مثلاً GEBCO)
gebco = ee.Image('NOAA/NGDC/ETOPO1').select('elevation')

# کلیپ کردن داده‌ها به منطقه مورد نظر
gebco_clipped = gebco.clip(geometry)

# ایجاد نقشه برای بررسی بصری
Map = geemap.Map(basemap='SATELLITE')
Map.centerObject(geometry, 6)
Map.addLayer(gebco_clipped, {'min': -6000, 'max': 0, 'palette': ['blue', 'green', 'white']}, 'GEBCO Bathymetry')
Map

# استخراج داده‌ها به‌عنوان گرید
# استفاده از geemap برای دانلود داده‌ها به‌صورت GeoTIFF
geemap.ee_export_image(
    gebco_clipped,
    filename='oman_sea_bathymetry.tif',
    scale=1000,  # رزولوشن تقریبی 1 کیلومتر
    region=geometry,
    file_per_band=False
)

# تبدیل GeoTIFF به فرمت NetCDF یا ASCII برای Delft3D
# خواندن فایل GeoTIFF با xarray
ds = xr.open_dataset('oman_sea_bathymetry.tif', engine='rasterio')

# ذخیره به‌صورت NetCDF
ds.to_netcdf('oman_sea_bathymetry.nc')

# یا ذخیره به‌صورت ASCII (برای Delft3D)
# استخراج آرایه داده‌ها
data = ds['elevation'].values
x = ds['x'].values
y = ds['y'].values

# ایجاد فایل ASCII
with open('oman_sea_bathymetry.asc', 'w') as f:
    f.write('ncols {}\n'.format(data.shape[1]))
    f.write('nrows {}\n'.format(data.shape[0]))
    f.write('xllcorner {}\n'.format(x.min()))
    f.write('yllcorner {}\n'.format(y.min()))
    f.write('cellsize {}\n'.format((x[1] - x[0])))
    f.write('NODATA_value -9999\n')
    np.savetxt(f, data, fmt='%f')

EEException: Image.select: Band pattern 'elevation' did not match any bands. Available bands: [bedrock, ice_surface]